In [1]:
# Import libraries
import ee
import geemap
import geedim as gd
import pandas as pd
import geopandas as gpd
import os

c:\ProgramData\miniconda3\envs\qgismaps2025\Lib\site-packages\pyproj\network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)


In [2]:
# Trigger the authentication flow
ee.Authenticate()

True

In [3]:
# Initialize the library
ee.Initialize()

# Initializes geemap/ GEE Python API
geemap.ee_initialize()

## 1. Functions for Layers Visualization

In [4]:
# Function to visualize a FeatureCollection
def feature_map(feature, vis_params, name):
    Map = geemap.Map(center=[8.64, 18.05], zoom =3)
    Map.add_basemap("SATELLITE")

    styled_layer = feature.style(**vis_params)
    
    Map.add_layer(styled_layer, {}, name)
    return Map


# Function to visualize a EE Image/Raster
def raster_map(raster, viz_params, name):
    """
    """

    Map = geemap.Map(center=[8.64, 18.05], zoom =3)
    Map.add_basemap("SATELLITE")
    Map.add_layer(raster, viz_params, name)
    return Map

## 2. Study Extent / Regions of Interest (ROIs)

In [ ]:
# Function to load boundary shapefile and convert to an Earth Engine Feature Collection (FC)
def boundary_to_fc(shp_path):

    # Convert shapefile to Earth Engine FeatureCollection
    fc = geemap.shp_to_ee(shp_path)

    return fc


# Path to the boundary shapefile
africa_shp_path = "D:/Lukumon_Projects/QGIS_Maps_2025/Africa_Pop_Distribution/Data/Bdry/Natural_Earth/ne_10m_admin_0_Africa_Dissolved.shp"

# Convert the boundary shapefile to Earth Engine FeatureCollections
roi = boundary_to_fc(africa_shp_path)

roi_map = feature_map(roi, {"color": "red", "fillColor": "00000000", "width": 2}, "Adm0 - Africa")
roi_map

Map(center=[8.64, 18.05], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

## 3. Download and Process/Aggregate Population Layer

In [20]:
# GHSL: Global Population Surfaces Image Collection
# GHSL: Spatial distribution of residential population. Population Count.
# Reference: https://developers.google.com/earth-engine/datasets/catalog/JRC_GHSL_P2023A_GHS_POP#description
ghsl_pop_collection = (ee.ImageCollection("JRC/GHSL/P2023A/GHS_POP")
            .filterDate("2020", "2025")
            .filterBounds(roi))


# Check the native projection of the GHS pop layer
ghsl_native_projection = ghsl_pop_collection.first().projection()
ghsl_native_resolution = ghsl_native_projection.nominalScale().getInfo()

print("The Native Projection of the GHSL Pop Layer: ", ghsl_native_projection.getInfo())
print("The Native Resolution/Scale of the GHSL Pop Layer: ", ghsl_native_resolution)

The Native Projection of the GHSL Pop Layer:  {'type': 'Projection', 'wkt': 'PROJCS["World_Mollweide", \n  GEOGCS["WGS 84", \n    DATUM["WGS_1984", \n      SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]], \n      AUTHORITY["EPSG","6326"]], \n    PRIMEM["Greenwich", 0.0], \n    UNIT["degree", 0.017453292519943295], \n    AXIS["Longitude", EAST], \n    AXIS["Latitude", NORTH]], \n  PROJECTION["Mollweide"], \n  PARAMETER["semi_minor", 6378137.0], \n  PARAMETER["false_easting", 0.0], \n  PARAMETER["false_northing", 0.0], \n  PARAMETER["central_meridian", 0.0], \n  UNIT["m", 1.0], \n  AXIS["Easting", EAST], \n  AXIS["Northing", NORTH]]', 'transform': [100, 0, -18041000, 0, -100, 9000000]}
The Native Resolution/Scale of the GHSL Pop Layer:  100


In [21]:
# Direct download of GHS pop data for specific year (2025)
ghsl_pop = ee.Image("JRC/GHSL/P2023A/GHS_POP/2025").clip(roi) # Change to your desired year. The data is available at 5-years interval.

# Inspect the date of the GHS layer
ghsl_pop_date = ee.Date(ghsl_pop.get("system:time_start")).format("YYYY-MM-dd").getInfo()
print(f"The date of the GHSL pop layer is {ghsl_pop_date}")

The date of the GHSL pop layer is 2025-01-01


In [22]:
# Aggregate the GHS population layer to desired lower resolution
def ghs_resolution_aggregate(pop_image, native_proj, new_resolution):

    """
    Aggregates a GHS population image to a lower spatial resolution using a specified scale.

    Parameters:
        pop_image (ee.Image): The GHS population image to be aggregated.
        native_proj (ee.Projection): The native projection of the input population image.
        new_resolution (float): The desired spatial resolution in meters (e.g., 500 or 1000).

    Returns:
        ee.Image: The population image aggregated to the specified lower resolution.
    
    Prints:
        The projection information and nominal scale of the aggregated image.
    """
    
    # Get the projection at the desired scale
    ghs_projection_at_new_resolution = native_proj.atScale(new_resolution)

    ghs_pop_at_new_res = pop_image.reduceResolution(
        reducer = ee.Reducer.sum().unweighted(),
        maxPixels = 1024
    ).reproject(
        crs = ghs_projection_at_new_resolution #Request the data at the scale and projection of reduced resolution (1km)
    )

    # Inspect the resolution/scale of the 'ghs_pop_at_new_res'
    ghs_pop_at_new_res_projection = ghs_pop_at_new_res.projection()
    ghs_pop_at_new_res_resolution = ghs_pop_at_new_res_projection.nominalScale().getInfo()

    print("The Native Projection of the Aggregated GHS Pop Layer: ", ghs_pop_at_new_res_projection.getInfo())
    print("The Resolution/Scale of the Aggregated GHS Pop Layer: ", ghs_pop_at_new_res_resolution)

    return ghs_pop_at_new_res 


# Apply function
ghsl_pop_500m = ghs_resolution_aggregate(ghsl_pop, ghsl_native_projection, 500)

The Native Projection of the Aggregated GHS Pop Layer:  {'type': 'Projection', 'wkt': 'PROJCS["World_Mollweide", \n  GEOGCS["WGS 84", \n    DATUM["WGS_1984", \n      SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]], \n      AUTHORITY["EPSG","6326"]], \n    PRIMEM["Greenwich", 0.0], \n    UNIT["degree", 0.017453292519943295], \n    AXIS["Longitude", EAST], \n    AXIS["Latitude", NORTH]], \n  PROJECTION["Mollweide"], \n  PARAMETER["semi_minor", 6378137.0], \n  PARAMETER["false_easting", 0.0], \n  PARAMETER["false_northing", 0.0], \n  PARAMETER["central_meridian", 0.0], \n  UNIT["m", 1.0], \n  AXIS["Easting", EAST], \n  AXIS["Northing", NORTH]]', 'transform': [500, 0, -18041000, 0, -500, 9000000]}
The Resolution/Scale of the Aggregated GHS Pop Layer:  500


In [ ]:
# Display the population layer
# Visualization parameters for the GHS pop layer
vis_params_ghs = {
  "bands" : ["population_count"],
  "min" : 0.0,
  "max": 50.0,
  "palette" : ["ffffe7", "FFc869", "ffac1d", "e17735", "f2552c", "9f0c21", "9f0c21"]
}

# Add the GHS pop layer to the map and visualize
pop_map = raster_map(ghsl_pop.clip(roi), vis_params_ghs , "GHSL Population 2025 (100m)")
pop_map.addLayer(ghsl_pop_500m.clip(roi), vis_params_ghs , "GHSL Population 2025 (500m)")
pop_map

## 4. Export The Population Layer 

In [ ]:
# Function to export image to Google Drive
def img_export_to_drive(image, desc, aoi, scale, folder_name):
    img_Export = ee.batch.Export.image.toDrive(
        image = image,
        description = desc,
        folder = folder_name,
        region = aoi,
        #crs = "EPSG:32634",
        scale = scale,
        maxPixels = 1e13
        )

    img_Export.start()
    
    return img_Export


# Export the Original GHSL population layer
#ghsl_pop_export = img_export_to_drive(ghsl_pop.clip(roi), "GHSL_Pop_Africa_2025_100m", roi, 100, "QGIS_Maps_2025")

# Export the GHSL population layer
ghsl_pop_export = img_export_to_drive(ghsl_pop_500m.clip(roi), "GHSL_Pop_Africa_2025_500m", roi.geometry(), 500, "QGIS_Maps_2025")

In [24]:
ghsl_pop_export.status()

{'state': 'RUNNING',
 'description': 'GHSL_Pop_Africa_2025_1km',
 'priority': 100,
 'creation_timestamp_ms': 1744035739238,
 'update_timestamp_ms': 1744036052010,
 'start_timestamp_ms': 1744035747658,
 'task_type': 'EXPORT_IMAGE',
 'attempt': 1,
 'batch_eecu_usage_seconds': 3667.877,
 'id': 'WVIFGITEWBCSXT6ZFAQJUQHP',
 'name': 'projects/earthengine-legacy/operations/WVIFGITEWBCSXT6ZFAQJUQHP'}